# Outils de création d'une carte avec la création d'un parcours
Fichier nécessaire: un fichier en format .txt contenant la liste des noms de lieux à recenser sur la carte.

Les outils de recherches d'informations permettent de récupérer les emplacements de chaque lieu (coordonnées GPS), le type de lieu dans lequel il est catégorisé ainsi qu'une brève description du lieu. Un fichier sous la forme d'un tableau lisible avec Excel est créé avec les informations récoltées.

La récupération des coordonnées GPS permet de retracer un parcours.

La carte est enregistrée en format html.

Il est possible à la fin d'ajouter, à la fin du notebook, son propre fond de carte.

**Avant de lancer le notebook**:

Créer un dossier sur son Google Drive intitulé "Creation-de-cartes".
Placer dans ce dossier ce notebook, ainsi que le fichier .txt contenant la liste des lieux.

In [ ]:
#@title Connexion à son compte GoogleDrive
#@markdown - Lancer la cellule
#@markdown - Cliquer sur le lien généré en dessous
#@markdown - Copier le mot de passe dans la nouvelle fenêtre ouverte où l'accès au compte est requis et le coller en dessous de la cellule.
!pip install geopy
!pip install --upgrade wptools
from google.colab import drive
import folium
from geopy.geocoders import Nominatim
import pandas as pd
from folium import plugins
import json
import urllib.parse, urllib.request
import numpy as np
import csv
from collections import Counter
from branca.element import Figure

drive.mount('/content/drive/')
%cd /content/drive/My Drive/NER-demo-lieux/

Requirement already up-to-date: wptools in /usr/local/lib/python3.7/dist-packages (0.4.17)
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/NER-demo-lieux


In [ ]:
#@title Récupération des identifiants Wikidata des lieux

def CallWikifier(text, lang=lang, threshold=seuil): 
  # Thresold = taux de sureté. 0.8 = sûr que le résultat est exact; mais peu de résultat.
  # 1 = tous les résultats mais certains faux résultats
    # Url de demande:
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", "4"),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
    # Appel de Wikifier 
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
        print(response)
    # Sortie des annotations:
#@markdown Changer ci-dessous le nom du fichie créé avec la liste des identifiants:
    nom_fichier_liste_identifiants = 'link-lieux2.txt' #@param {type:"string"}
    with open(nom_fichier_liste_identifiants, 'w') as outfile:
        for annotation in response["annotations"]:
          ## Récupération (uniquement de l'ID wikidata):
          itemid = annotation["wikiDataItemId"] 
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)
#@markdown Changer la langue si nécessaire (par défaut: français):
lang = "fr" #@param {type:"string"}
#@markdown Si la détéction n'est pas assez précise, descendre le seuil à 0.8 (moins de noms vont être identifiés), dans la cas contraire, monter le seuil à 1.
seuil = 0.8 #@param {type:"string"}
#@markdown Changer ci-dessous le nom du fichier avec la liste des lieux à détecter:
fichier_avec_liste_des_lieux = "test-it-lieux.txt" #@param {type:"string"}
with open (fichier_avec_liste_des_lieux, "r") as myfile:
    liste=myfile.read()
CallWikifier(text=liste, lang="fr")

{'annotations': [{'title': 'Place (voie)', 'url': 'http://fr.wikipedia.org/wiki/Place_(voie)', 'lang': 'fr', 'pageRank': 0.01030158054090963, 'secLang': 'en', 'secTitle': 'Town square', 'secUrl': 'http://en.wikipedia.org/wiki/Town_square', 'wikiDataItemId': 'Q174782', 'wikiDataClasses': [], 'dbPediaTypes': [], 'dbPediaIri': 'http://dbpedia.org/resource/Town_square', 'supportLen': 3, 'support': [{'wFrom': 0, 'wTo': 0, 'chFrom': 0, 'chTo': 4, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0004208193773132968, 'prbConfidence': 0.9652415111552961, 'entropy': 0.05889157293723925}, {'wFrom': 10, 'wTo': 10, 'chFrom': 61, 'chTo': 65, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0004208193773132968, 'prbConfidence': 0.9652415111552961, 'entropy': 0.05889157293723925}, {'wFrom': 17, 'wTo': 17, 'chFrom': 105, 'chTo': 109, 'pMentionGivenSurface': 0.01711017953157751, 'pageRank': 0.0004208193773132968, 'prbConfidence': 0.9652415111552961, 'entropy': 0.05889157293723925}]},

In [ ]:
#@title Récupération des informations de chaque lieu reconnu et lié à wikidata:
%cd /content/drive/My Drive/NER-demo-lieux/

#@markdown Toutes les informations (le type de lieu, les coordonnées GPS (latitude et longitude) ainsi qu'une brève description du lieu) sont enregistrées et exportées sous la forme d'un fichier de données JSON ainsi que sous la forme d'un tableau dans un fichier Excel dans le Google Drive.


import wptools
import re
import itertools

donnees ={}
nom = []
type_lieu = []
latitude = []
longitude = []
longitude = []
description = []
#@markdown Indiquer le nom du fichier avec les identifiants créé dans la cellule précédente:
nom_fichier_liste_identifiants = "link-lieux2.txt" #@param {type:"string"}
#@markdown Indiquer le nom du fichier en format JSON créé avec les informations sur les lieux:
fichier_json_infos_lieux = "donnees_lieux.json" #@param {type:"string"}
# Utilisation du document créé précédement avec les identifiants wikidata:
with open(nom_fichier_liste_identifiants, "r") as a_file:
      with open(ficher_json_infos_lieux, "w") as fichier : 
        for line in a_file:
            for word in line.split():
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
## Pour avoir toutes les informations:
              try:
                infobox=page.data['infobox']
                nom = infobox.get("nom")
                type_lieu=infobox.get("type")
                if type_lieu is None: 
                  try:
                    type_lieu=page.data['what']
                  except KeyError as ke:
                    pass
                latitude = infobox.get("latitude")
                longitude = infobox.get("longitude")
                description = page.data["description"]
              except AttributeError as at:
                print("Ce n'est pas un lieu!")
# Création d'un dictionnaire + fichier json avec les informations récupérées sur les lieux:
              try:
                donnees[nom] = {"Type de lieu" : type_lieu,
                              "Latitude" : latitude, "Longitude" : longitude, "Description": description}
              except TypeError as tpy:
                pass
      
        json.dump(donnees, fichier, ensure_ascii=False)

#@markdown Indiquer le nom du fichier en format XLSX créé avec les informations sur les lieux:
fichier_xlsx_infos_lieux = "informations_lieux.xlsx" #@param {type:"string"}


# Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel:
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel(fichier_xlsx_infos_lieux)

In [ ]:
#@title Outils de calculs statistiques

#@markdown Indiquer le nom du fichier en format JSON créé précédement avec les informations sur les lieux:
fichier_json_infos_lieux = "donnees_lieux.json" #@param {type:"string"}

# Récupération et création d'une liste avec catégories de lieux
keys = donnees[nom]["Type de lieu"]
liste_des_types = []
with open(fichier_json_infos_lieux, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_types=value["Type de lieu"]
      liste_des_types.append(test_types)


def proportion_types(liste):
  c = Counter(liste)
  print("Voici le nombre de lieux, triés par chaque type:", c) ## Affiche le nombre de lieux de chaque type
  prop = [(i, c[i] / len(liste) * 100.0) for i in c]
  print("Voici la proportion de chaque type de lieux:", prop) ## Affiche la proportion de chaque type de lieux
  
proportion_types(liste_des_types)



Voici le nombre de lieux, triés par chaque type: Counter({'capitale': 1, None: 1, 'taxon': 1, 'station de métro': 1, 'Château fort, arsenal et prison': 1, '[[Boisson énergisante]] [[psychotrope]] [[stimulant]]e': 1, 'commune de France': 1})
Voici la proportion de chaque type de lieux: [('capitale', 14.285714285714285), (None, 14.285714285714285), ('taxon', 14.285714285714285), ('station de métro', 14.285714285714285), ('Château fort, arsenal et prison', 14.285714285714285), ('[[Boisson énergisante]] [[psychotrope]] [[stimulant]]e', 14.285714285714285), ('commune de France', 14.285714285714285)]


In [ ]:
#@title Création d'une carte avec parcours:
keys_lat = donnees[nom]["Latitude"]
keys_long = donnees[nom]["Longitude"]
liste_des_lat = []
liste_des_long = []
#@markdown Indiquer le nom du fichier en format JSON créé précédement avec les informations sur les lieux:
fichier_json_infos_lieux = "donnees_lieux.json" #@param {type:"string"}
with open(fichier_json_infos_lieux, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_lat=value["Latitude"]
      liste_des_lat.append(test_lat)
      test_long=value["Longitude"]
      liste_des_long.append(test_long)

coords = []
for ju, ij in zip(liste_des_lat, liste_des_long):
  if ju is None:
    pass  
  elif ij is None:
    pass
  else:
    coords.append([float(ju), float(ij)])
print(coords) ### Crée un array de coordonnées en supprimant les None

fig5=Figure(height=550,width=750)
m5=folium.Map(location=[48.856614, 2.3522219], zoom_start=12)
fig5.add_child(m5)

f1=folium.FeatureGroup("Parcours")
line_1=folium.vector_layers.PolyLine(coords,popup='<b>Parcours_test</b>',tooltip='Tour de France',color='blue',weight=4).add_to(f1)

f1.add_to(m5)
folium.LayerControl().add_to(m5)
m5

1000


In [ ]:
#@title Créer et ajouter son fond de carte
#@markdown Si l'on veut ajouter son propre fond de carte personnalisé:
#@markdown - Création d’un compte sur : https://mapwarper.net/
#@markdown - Téléchargement du fond de carte (IIIF recommandé, avec la meilleure résolution possible)
#@markdown - Sur Map Warper: dans l’onglet « Upload Map » il est possible de charger le fond de carte que l’on veut utiliser, et indiquer toutes les métadonnées nécessaires.
#@markdown - Dans l’onglet "Rectify": il faut indiquer le plus de points identiques sur les deux cartes pour permettre la superposition du fond de carte chargé sur des données géolocalisées. Cliquer sur un point sur chaque carte, puis cliquer sur « add Control point » en dessous des cartes. Répéter l’opération le plus de fois possible pour obtenir un bon résultat. Dans l'encadré en dessous des deux fonds de carte, cliquer sur "Warp image": le fond s'ajoute. Si le résultat n'est pas satisfaisant, il possbile d'ajouter d'autres repères pour améliorer la qualité du nouveau fond de carte créé.
#@markdown - Se rendre dans l'onglet "Export". Dans "Map services", copier le lien entier se situant après "Tiles (Google/OSM scheme):", débutant par http et se terminant par .png
#@markdown - Coller ensuite le lien ci-dessous:
fond_de_carte_cree = "https://mapwarper.net/maps/tile/54525/{z}/{x}/{y}.png" #@param {type:"string"}

fig5=Figure(height=550,width=750)
m5=folium.Map(location=[48.856614, 2.3522219], zoom_start=12,
                 tiles=fond_de_carte_cree, attr="None")

keys_lat = donnees[nom]["Latitude"]
keys_long = donnees[nom]["Longitude"]
liste_des_lat = []
liste_des_long = []
fichier_json = "/content/drive/My Drive/NER-demo-lieux/donnees_lieux.json"
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_lat=value["Latitude"]
      liste_des_lat.append(test_lat)
      test_long=value["Longitude"]
      liste_des_long.append(test_long)

coords = []
for ju, ij in zip(liste_des_lat, liste_des_long):
  if ju is None:
    pass  
  elif ij is None:
    pass
  else:
    coords.append([float(ju), float(ij)])
print(coords) ### Crée un array de coordonnées en suppirimant les None

fig5.add_child(m5)

f1=folium.FeatureGroup("Parcours")
line_1=folium.vector_layers.PolyLine(coords,popup='<b>Parcours_test</b>',tooltip='Tour de France',color='blue',weight=4).add_to(f1)

f1.add_to(m5)
folium.LayerControl().add_to(m5)
m5

[[48.856578, 2.351828], [48.831483, 2.355692], [48.853333, 2.369167], [48.8477777778, 2.43916666667]]
